# 統計解析

In [ ]:
# このセルはそのまま実行して、tdquiz コマンドをロードしてください。
# tdquiz コマンドはクエリを実行してその結果を返します。
%load_ext tdquiz

# tdquiz コマンドをテスト
%tdquiz SELECT USER, SESSION, CURRENT_TIMESTAMP

<h3>前年比増加率</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">arithmetic-function</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">logarithm</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">math</span>
<br>
増加率はいくつかの定義があります。たとえば次のようなものがよく使われます。
<ol>
<li> y<sub>t</sub> / y<sub>t-1</sub> - 1
<li> 1 - y<sub>t-1</sub> / y<sub>t</sub>
<li> log<sub>e</sub>(y<sub>t</sub>) - log<sub>e</sub>(y<sub>t-1</sub>)
</ol>
1つ目は前期の値を基準に変化の割合を計算するもので、最も一般的です。
2つ目は基準を来期にしたものです。
3つ目は自然対数を利用した定義で、各期について対象的である特徴があります。
<br />
これらの定義は変化が大きすぎなければ近い値を取ります。
<br />
TeraShirt社の月次売上について、2021年から2022へかけての前年同月比での増加率を、3つの定義で計算してください。
なお、いずれかの年の売上が存在しない場合は除いてください。

<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">with tmp AS (
SELECT
  EXTRACT(MONTH FROM purchase_date) AS sales_month,
  EXTRACT(YEAR FROM purchase_date) AS sales_year,
  SUM(sales_value) AS revenue
FROM
  TeraShirt.store_sales
WHERE
  sales_year IN (2021, 2022)
GROUP BY 1,2
)
SELECT
  COALESCE(a.sales_month, b.sales_month) AS "month",
  1.00 * b.revenue / a.revenue - 1 AS inc_rate1,
  1 - 1.00 * a.revenue / b.revenue AS inc_rate2,
  LN(b.revenue) - LN(a.revenue) AS inc_rate3
FROM 
  tmp AS a
  INNER JOIN tmp AS b
    ON a.sales_month = b.sales_month
       AND a.sales_year = 2021 AND b.sales_year = 2022
ORDER BY 1</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */






<h3>購入頻度の比較</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">median</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">aggregation</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">window-function</span>
<br>
TeraShirt社では、会員顧客を商品の購入頻度でグループ分けして、個別施策を講じることを検討しています。
そこで、会員ごとに連続する購入日の間の日数間隔について調べることになりました。
ただし、同じ日に起こった購入は計算上は1度の購入と考えます。
<br />
顧客間での購入頻度を比べるために、顧客ごとにその購入回数、購入間隔の平均値と中央値を算出してください。
最終的には、10回以上の購入があった人に限定し、中央値の小さい順に並べてください (中央値が同じ場合は平均値、それも同じなら顧客IDでいずれも昇順に並べてください)。
出力列は、顧客ID、顧客名 (名字と名前を半角スペースで結合)、購入間隔の平均、購入間隔の中央値としてください。
<br />
中央値の小さい顧客ほど頻度の高い顧客ということになります。

<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">WITH tmp AS (
  SELECT
    purchase_date,
    purchase_date - LAG(purchase_date) OVER (PARTITION BY customer_id ORDER BY purchase_date) AS duration,
    customer_id
  FROM
    ( SELECT DISTINCT purchase_date, customer_id FROM terashirt.store_sales WHERE customer_id IS NOT NULL ) AS a
)
SELECT
  b.customer_id,
  b.last_name || ' ' || b.first_name AS customer_name,
  count(*) AS n_visit,
  avg(duration) AS average_days,
  median(duration) AS median_days
FROM
  tmp
  INNER JOIN terashirt.customer AS b ON tmp.customer_id = b.customer_id
GROUP BY 1,2
HAVING count(*) >= 10
ORDER BY 5,4,1</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */






<h3>購入間隔分布の比較</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">survival-analysis</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">cumulative-distribution</span>
<br>
TeraShirt社では、個人別の購入間隔の分布を可視化したいと考えています。モデルケースとして、顧客番号2と54の2名について考えます。
<br />
各個人について購入間隔の日数を求め（同じ日の購入は1回のみカウント）、その各日数に対して「購入間隔がその日数以下であった割合」を計算してください。
なお、割合についてはFLOAT型としてください。
<br />
出力するのは、顧客ID、購入間隔日数、購入間隔がその日数以下であった割合の3列として、顧客IDと購入間隔日数で並べ替えてください。
<br />
Optional: 結果を用いて、個人ごとに横軸に購入間隔日数、縦軸に割合を取ってグラフ化してみてください。
これは、統計学でいう累積分布関数になります。このグラフから、2人の顧客の購入間隔の大小はどう比べられるでしょうか？

<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">with tmp AS (
  SELECT DISTINCT customer_id, purchase_date
  FROM TeraShirt.store_sales
  WHERE customer_id IN (2, 54)
)
,tmp2 AS (
  SELECT
    customer_id,
    purchase_date - LAG(purchase_date) OVER (
    PARTITION BY customer_id ORDER BY purchase_date
  ) AS duration
  FROM tmp
)
,tmp3 AS (
  SELECT customer_id, duration, COUNT(*) AS freq
  FROM tmp2
  WHERE duration IS NOT NULL
  GROUP BY 1,2
)
SELECT
  customer_id, duration,
  CAST(1 AS FLOAT) * SUM(freq) OVER (
    PARTITION BY customer_id
    ORDER BY duration DESC
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) / SUM(freq) OVER (PARTITION BY customer_id)
  AS cum_ratio
FROM
 tmp3
ORDER BY 1,2</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */




